In [1]:
from numpy import log2, sqrt, log
from scipy.special import comb
from AllFunctions import *
from random import sample
import time
import matplotlib.pyplot as plt

In [2]:
class RandomForest(DecisionTree):
    def __init__(self, train, labels, attributes, depth = -1, weights = None):
        super().__init__(train, labels, attributes, depth, weights)

    
    def best_att(self, train, labels, attributes, weights = None):
    
        #k = int(sqrt(len(attributes)))
        if len(attributes)>= 2:
            k= 2
        else:
            k=1
            
        sample_attributes = sample(attributes, k)
        
        lable_Ent = entropy(labels, weights)
        Max = -1
        Best = None
        Best_values = None
    
        for attribute in sample_attributes: 
            temp, temp_values = Entropy_given_attribute(train, labels, attribute, weights) 
            if lable_Ent - temp >  Max:
                Max = lable_Ent - temp
                Best = attribute
                Best_values = temp_values
                    
        return(Best, Best_values)

In [3]:
train, train_labels, test, test_labels = load_bank()


for i in range(len(train_labels)):
    if train_labels[i] =='no':
        train_labels[i] = -1
    elif train_labels[i] =='yes':
        train_labels[i] = 1
        
    
    if test_labels[i] =='no':
        test_labels[i] = -1
    elif test_labels[i] =='yes':
        test_labels[i] = 1

# Random Forest

In [4]:
start = time.time()

Forest = []
for x in range(200):
    X, Y = my_sample(train, train_labels)
    Forest.append (RandomForest(X, Y, [i for i in range(len(train[0]))], depth = -1))
    
print(time.time() - start)

85.90065002441406


In [5]:
def Error(List_Tress, X, Y):
    
    n = len(Y)
    error = [0] * len(List_Tress)
    
    for i in range(n):
        count = {}
        majority = None
        Max = -1
        for j in range(len(List_Tress)):
            

            temp = List_Tress[j].predict(X[i]) 
            if temp not in count:
                count[temp] = 0
                
            count[temp] += 1
            if count[temp] > Max:
                Max = count[temp]
                majority = temp
        
            if majority != Y[i]:
                error[j] += 1/len(Y)
    return(error)

# Bagged Decision Trees

In [6]:
def Bagged_Trees(train_data, training_labels, n_iter, n_samples = None):
    #start = time.time()
    Forest = []

    for x in range(n_iter):
        X, Y = my_sample(train_data, training_labels, n_samples)
        Forest.append (DecisionTree(X, Y, [i for i in range(len(train[0]))], depth = -1))
    
    return(Forest)

# Bias and Variance Decomposition

In [7]:
start = time.time()
Bagged_predictors = []

for i in range(20):
    
    L = sample([x for x in range(len(train_labels))], 1000) # sample 1000 indices without replacement
    X = [train[i] for i in L]
    Y = [train_labels[i] for i in L]
    
    Bagged_predictors.append(Bagged_Trees(X, Y, n_iter = 200)) 
print(time.time() - start)

345.5914659500122


# 1. Computing Bias and Variance using the 100 single trees

In [8]:
def bias_variance(Bagged_predictors, x, y):    
    summ1 = 0
    summ2 = 0
    for i in range(len(Bagged_predictors)):
        temp = Bagged_predictors[i][0].predict(x)
        summ1 += temp
        summ2 += temp**2
        
    return (y - summ1/len(Bagged_predictors))**2, summ2/len(Bagged_predictors) - (summ1/len(Bagged_predictors))**2

In [9]:
Bias = 0
Variance = 0
for i in range(len(test_labels)):
    b, v = bias_variance(Bagged_predictors, test[i], test_labels[i])
    Bias += b/len(test_labels)
    Variance += v/len(test_labels)    

In [15]:
print(' Bias: {}\n Variance: {}\n Bias + Variance: {}'.format(Bias, Variance, Bias + Variance))

 Bias: 0.35061800000000254
 Variance: 0.34134199999999715
 Bias + Variance: 0.6919599999999997


# 2. Computing Bias and Variance using the 100 Bagged predictors

In [11]:
def B_predictor(List_Tress, x):
    
    count = {}
    majority = None
    Max = -1
    for j in range(len(List_Tress)):
        temp = List_Tress[j].predict(x) 
        if temp not in count:
            count[temp] = 0
                
        count[temp] += 1
        if count[temp] > Max:
            Max = count[temp]
            majority = temp
    return(majority)

In [12]:
def new_bias_variance(Bagged_predictors, x, y):    
    summ1 = 0
    summ2 = 0
    for i in range(len(Bagged_predictors)):
        
        temp = B_predictor(Bagged_predictors[i], x)
        summ1 += temp
        summ2 += temp**2
        
    return (y - summ1/len(Bagged_predictors))**2, summ2/len(Bagged_predictors) - (summ1/len(Bagged_predictors))**2

In [13]:
B_Bias = 0
B_Variance = 0
start= time.time()
for i in range(len(test_labels)):
    b, v = new_bias_variance(Bagged_predictors, test[i], test_labels[i])
    B_Bias += b/len(test_labels)
    B_Variance += v/len(test_labels)
print(time.time()-start)

146.56753587722778


In [14]:
print(' Bagged_Bias: {}\n Bagged_Variance: {}\n Bagged_Bias + Bagged_Variance: {}'.format(B_Bias, B_Variance, B_Bias + B_Variance))

 Bagged_Bias: 0.3524180000000025
 Bagged_Variance: 0.20194200000000023
 Bagged_Bias + Bagged_Variance: 0.5543600000000027
